In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  2


In [2]:
import json 
json_dir = '/root/notebooks/0858611-2/tooth_xray/model/all_tooth_data_set/mapping.json'
with open(json_dir) as json_file:
    json_data = json.load(json_file)
    

In [3]:
def get_level(mean):
    if mean>=0 and mean<=2:
        return 'normal'
    elif mean>=3 and mean<=4:
        return 'medium'
    elif mean>4:
        return 'serious'
    else:
        return 'over_range'

In [4]:
from statistics import mean
import pandas as pd
img_path = []
img_cal = []
img_cal_mean = []
img_cal_level = []

for key in json_data:
    try:
        new_path = key.replace("Dataset/Images", "/root/notebooks/0858611-2/tooth_xray/model/all_tooth_data_set/Images")
        mean_cal = int(mean(json_data[key]))
        img_cal_level.append(get_level(mean_cal))
        img_cal_mean.append(str(mean_cal))
        img_path.append(new_path)
        img_cal.append(json_data[key])
    except:
        print(key,json_data[key])
        pass
    

Dataset/Images/037621_12_.png None
Dataset/Images/037621_13_.png None
Dataset/Images/037621_26_.png None
Dataset/Images/037621_25_.png None
Dataset/Images/039462_14_.png None
Dataset/Images/039462_22_.png None
Dataset/Images/039462_21_.png None


In [5]:
print(len(img_path))
print(len(img_cal))
print(len(img_cal_mean))
print(len(img_cal_level))
print(img_cal_level.count('normal'))
print(img_cal_level.count('medium'))
print(img_cal_level.count('serious'))

1506
1506
1506
1506
699
346
109


In [6]:
import pandas as pd

dict = {"image_path": img_path,  
        "img_cal": img_cal,
        "img_cal_mean":img_cal_mean,
        "img_cal_level":img_cal_level
       }

tooth_dataframe = pd.DataFrame(dict)
print(tooth_dataframe.head())

                                          image_path img_cal img_cal_mean  \
0  /root/notebooks/0858611-2/tooth_xray/model/all...  [5, 3]            4   
1  /root/notebooks/0858611-2/tooth_xray/model/all...  [3, 4]            3   
2  /root/notebooks/0858611-2/tooth_xray/model/all...  [7, 9]            8   
3  /root/notebooks/0858611-2/tooth_xray/model/all...  [4, 5]            4   
4  /root/notebooks/0858611-2/tooth_xray/model/all...  [4, 3]            3   

  img_cal_level  
0        medium  
1        medium  
2       serious  
3        medium  
4        medium  


In [7]:
tooth_dataframe = tooth_dataframe[tooth_dataframe.img_cal_level != 'over_range']

normal_dataframe = tooth_dataframe[tooth_dataframe.img_cal_level == 'normal']
medium_dataframe = tooth_dataframe[tooth_dataframe.img_cal_level == 'medium']
serious_dataframe = tooth_dataframe[tooth_dataframe.img_cal_level == 'serious']

In [8]:
#img to array and resize 
import cv2
import numpy as np

def imgpath_list_to_array_list(imgpath_list,imgsize=(256,256)):
    array_list = []
    for img_path in imgpath_list:
        img = cv2.imread(img_path)
        img = cv2.resize(img,imgsize)
        #img = (img-127.5)/127.5
        array_list.append(img)
    return np.array(array_list)

def label_list_to_num(train_label):
    label_array = []
    for symptom in train_label:
        if symptom == 'normal':
            label_array.append(0)
        if symptom == 'medium':
            label_array.append(1)
        if symptom == 'serious':
            label_array.append(2)
    return np.array(label_array)

train_data_array = imgpath_list_to_array_list(tooth_dataframe.image_path)
train_label_array = label_list_to_num(tooth_dataframe.img_cal_level)

normal_data_array = imgpath_list_to_array_list(normal_dataframe.image_path)
normal_label_array = label_list_to_num(normal_dataframe.img_cal_level)
medium_data_array = imgpath_list_to_array_list(medium_dataframe.image_path)
medium_label_array= label_list_to_num(medium_dataframe.img_cal_level)
serious_data_array = imgpath_list_to_array_list(serious_dataframe.image_path)
serious_label_array = label_list_to_num(serious_dataframe.img_cal_level)


print(train_data_array.shape)
print(train_label_array.shape)

(1154, 256, 256, 3)
(1154,)


In [9]:
from tensorflow.keras import Model
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input,Dropout


def my_convolution_model(input_shape, classes):
    inputs=Input(input_shape)
    x = Conv2D(32, (3, 3), activation='relu')(inputs)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(256, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Flatten()(x)
    x = Dense(2048, activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(classes)(x)
    model = Model(inputs, x)
    model.compile(optimizer='adam',
          loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
          metrics=['accuracy'])
    return model

input_shape= (256,256,3)
classes    = 3
batch_size = 256
epochs     = 30

mirrored_strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])
with mirrored_strategy.scope():
    model_convolution = my_convolution_model(input_shape, classes)

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


In [10]:
#model_convolution.summary()

In [11]:
history = model_convolution.fit(train_data_array[:1000], train_label_array[:1000], epochs=epochs, 
                     validation_data=(train_data_array[1000:], train_label_array[1000:]),shuffle=True,batch_size = batch_size)

Train on 1000 samples, validate on 154 samples
Epoch 1/30
INFO:tensorflow:batch_all_reduce: 16 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:batch_all_reduce: 16 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then br

In [12]:
modelpredict = my_convolution_model(input_shape, classes)

weights = model_convolution.get_weights()
modelpredict.set_weights(weights)

In [13]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model_convolution.evaluate(train_data_array[1000:],  train_label_array[1000:], verbose=2)

154/1 - 1s - loss: 1.8123 - accuracy: 0.5909


In [14]:
model_convolution.evaluate(train_data_array[1000:],  train_label_array[1000:], verbose=2)

154/1 - 1s - loss: 1.8123 - accuracy: 0.5909


[2.171412365777152, 0.59090906]

In [15]:
print('following is recall rate')
print('\n# Evaluate on normal_data')
model_convolution.evaluate(normal_data_array,normal_label_array, verbose=2)

print('\n# Evaluate on medium_data')
model_convolution.evaluate(medium_data_array,medium_label_array, verbose=2)

print('\n# Evaluate on serious_data')
model_convolution.evaluate(serious_data_array,serious_label_array, verbose=2)

following is recall rate

# Evaluate on normal_data
699/1 - 1s - loss: 0.8040 - accuracy: 0.9599

# Evaluate on medium_data
346/1 - 1s - loss: 1.0510 - accuracy: 0.9306

# Evaluate on serious_data
109/1 - 1s - loss: 3.6978 - accuracy: 0.7798


[1.7999304325208751, 0.7798165]

In [16]:
#prediction 

In [17]:
from sklearn.metrics import confusion_matrix
import pandas as pd

def recall_accuracy_rate(test_data_array,test_label_array,model):
    model_predict =[]
    for img in test_data_array:
        reshape = img.reshape(1,256,256,3)
        image = tf.cast(reshape, tf.float32)
        model_predict.append(np.argmax(model.predict(image)))
        
    matrix = confusion_matrix(test_label_array, model_predict)
    matrix_split = np.split(matrix,3,axis=1)
    print('column:* is prediction result')
    print('row*: is real class')
    dict = {#"": img_path,
            "class:0":matrix_split[0].reshape(3),
            "class:1":matrix_split[1].reshape(3),
            "class:2":matrix_split[2].reshape(3)
           }
    
    predict_table = pd.DataFrame(dict)
    return predict_table,matrix

In [19]:
def precision(label, confusion_matrix):
    col = confusion_matrix[:, label]
    return confusion_matrix[label, label] / col.sum()
    
def recall(label, confusion_matrix):
    row = confusion_matrix[label, :]
    return confusion_matrix[label, label] / row.sum()

In [20]:
predict_table,matrix = recall_accuracy_rate(train_data_array[1000:],train_label_array[1000:],modelpredict)
for i in range(len(predict_table.columns)):
    p = precision(i,matrix)
    r = recall(i,matrix) 
    print('The {} precision is {:.4f} and recall is {:.4f} in '.format(predict_table.columns[i],p,r))

column:* is prediction result
row*: is real class
The class:0 precision is 0.7642 and recall is 0.7570 in 
The class:1 precision is 0.1463 and recall is 0.3158 in 
The class:2 precision is 0.5714 and recall is 0.1429 in 


In [21]:
predict_table.head()

,class:0,class:1,class:2
0,81,24,2
1,12,6,1
2,13,11,4
